In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action="ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import data
train = pd.read_csv(dirname + "/train.csv",parse_dates=['date'],index_col=['date'])
test = pd.read_csv(dirname  + "/test.csv"  ,parse_dates=['date'],index_col=['date'])
submission = pd.read_csv(dirname + "/sample_submission.csv")

# Create some of the date variables
train['day'] = train.index.day
train['month'] = train.index.month
train['year'] = train.index.year
train['dayofweek'] = train.index.dayofweek
train['weekofyear']  = train.index.isocalendar().week
train['is_weekend'] = train.index.dayofweek // 5

test['day'] = test.index.day
test['month'] = test.index.month
test['year'] = test.index.year
test['dayofweek'] = test.index.dayofweek
test['weekofyear']  = test.index.isocalendar().week
test['is_weekend'] = test.index.dayofweek // 5

In [ ]:
rf = RandomForestRegressor(n_estimators=50, min_samples_leaf = 7, random_state=123)

# Add weights to most recent data
train_main = pd.concat([train,
                        train.loc[train.index > train.index.max()- pd.DateOffset(30, 'D')],
                        train.loc[train.index > train.index.max()- pd.DateOffset(30, 'D')],
                        train.loc[train.index > train.index.max()- pd.DateOffset(15, 'D')]
                       ])

# Train a model
rf.fit(X=train_main[['store', 'item','year','month','dayofweek','is_weekend']], y=train_main['sales'])

# Get predictions for the test set
test['sales'] = rf.predict(test[['store', 'item','year','month','dayofweek','is_weekend']])

test[['id', 'sales']].to_csv("submission.csv", index=False)